# DMU0: IAC SDSS S82 extraction

The original extraction was missing objects below RA=0. In this notebook we combine the individual catalogues that were downloaded from ftp.iac.es and also available from http://www.iac.es/proyecto/stripe82/pages/data.php into one full catalogue to be used here and uploaded to CDS.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
017bb1e (Mon Jun 18 14:58:59 2018 +0100) [with local modifications]
This notebook was executed on: 
2018-09-20 15:34:57.807622


In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, vstack, join
import numpy as np
from pymoc import MOC

import glob

## I - Get list of files
Merge stars and galaxies into one table

In [ ]:
file_list = glob.glob('./data/tiles/*.cat')
file_list[:10]

In [ ]:

test_gal = Table.read('./data/tiles/f0011_gal.cat', format='ascii')
test_star = Table.read('./data/tiles/f0011_ps.cat', format='ascii')

In [ ]:
test_gal[:10]

In [ ]:
test_star[:10]

In [ ]:
columns = {

    1:  'ALPHA_J2000'          ,#Right ascension of barycenter (J2000)                    [deg]
    2:  'DELTA_J2000'          ,#Declination of barycenter (J2000)                        [deg]
    3:  'MAG_AUTO_u'           ,#Sloan u | Kron-like elliptical aperture magnitude        [mag]
    4:  'MAG_PETRO_u'          ,#Sloan u | Petrosian-like elliptical aperture magnitude   [mag]
    5:  'MAG_APER_1_u'         ,#Sloan u | 5 pixel fixed aperture magnitude               [mag]
    6:  'MAG_APER_2_u'         ,#Sloan u | 10 pixel fixed aperture magnitude              [mag]
    7:  'MAG_APER_3_u'         ,#Sloan u | 20 pixel fixed aperture magnitude              [mag]
    8:  'MAG_APER_4_u'         ,#Sloan u | 30 pixel fixed aperture magnitude              [mag]
    9:  'MAGERR_AUTO_u'        ,#Sloan u | RMS error for AUTO magnitude                   [mag]
    10: 'MAGERR_PETRO_u'       ,#Sloan u | RMS error for PETROsian magnitude              [mag]
    11: 'MAGERR_APER_1_u'      ,#Sloan u | RMS error for MAG_APER (1)                     [mag]
    12: 'MAGERR_APER_2_u'      ,#Sloan u | RMS error for MAG_APER (2)                     [mag]
    13: 'MAGERR_APER_3_u'      ,#Sloan u | RMS error for MAG_APER (3)                     [mag]
    14: 'MAGERR_APER_4_u'      ,#Sloan u | RMS error for MAG_APER (4)                     [mag]
    15: 'FLUX_RADIUS_u'        ,#Sloan u | Fraction-of-light radii (0.5 of total)         [pixel] 
    16: 'MAG_AUTO_g'           ,#Sloan g | Kron-like elliptical aperture magnitude        [mag] 
    17: 'MAG_PETRO_g'          ,#Sloan g | Petrosian-like elliptical aperture magnitude   [mag] 
    18: 'MAG_APER_1_g'         ,#Sloan g | 5 pixel fixed aperture magnitude               [mag] 
    19: 'MAG_APER_2_g'         ,#Sloan g | 10 pixel fixed aperture magnitude              [mag] 
    20: 'MAG_APER_3_g'         ,#Sloan g | 20 pixel fixed aperture magnitude              [mag] 
    21: 'MAG_APER_4_g'         ,#Sloan g | 30 pixel fixed aperture magnitude              [mag] 
    22: 'MAGERR_AUTO_g'        ,#Sloan g | RMS error for AUTO magnitude                   [mag] 
    23: 'MAGERR_PETRO_g'       ,#Sloan g | RMS error for PETROsian magnitude              [mag] 
    24: 'MAGERR_APER_1_g'      ,#Sloan g | RMS error for MAG_APER (1)                     [mag] 
    25: 'MAGERR_APER_2_g'      ,#Sloan g | RMS error for MAG_APER (2)                     [mag] 
    26: 'MAGERR_APER_3_g'      ,#Sloan g | RMS error for MAG_APER (3)                     [mag] 
    27: 'MAGERR_APER_4_g'      ,#Sloan g | RMS error for MAG_APER (4)                     [mag] 
    28: 'FLUX_RADIUS_g'        ,#Sloan g | Fraction-of-light radii (0.5 of total)         [pixel] 
    29: 'MAG_AUTO_r'           ,#Sloan r | Kron-like elliptical aperture magnitude        [mag]  
    30: 'MAG_PETRO_r'          ,#Sloan r | Petrosian-like elliptical aperture magnitude   [mag]  
    31: 'MAG_APER_1_r'         ,#Sloan r | 5 pixel fixed aperture magnitude               [mag]  
    32: 'MAG_APER_2_r'         ,#Sloan r | 10 pixel fixed aperture magnitude              [mag]  
    33: 'MAG_APER_3_r'         ,#Sloan r | 20 pixel fixed aperture magnitude              [mag]  
    34: 'MAG_APER_4_r'         ,#Sloan r | 30 pixel fixed aperture magnitude              [mag]  
    35: 'MAGERR_AUTO_r'        ,#Sloan r | RMS error for AUTO magnitude                   [mag]  
    36: 'MAGERR_PETRO_r'       ,#Sloan r | RMS error for PETROsian magnitude              [mag]  
    37: 'MAGERR_APER_1_r'      ,#Sloan r | RMS error for MAG_APER (1)                     [mag]  
    38: 'MAGERR_APER_2_r'      ,#Sloan r | RMS error for MAG_APER (2)                     [mag]  
    39: 'MAGERR_APER_3_r'      ,#Sloan r | RMS error for MAG_APER (3)                     [mag]  
    40: 'MAGERR_APER_4_r'      ,#Sloan r | RMS error for MAG_APER (4)                     [mag]  
    41: 'FLUX_RADIUS_r'        ,#Sloan r | Fraction-of-light radii (0.5 of total)         [pixel]
    42: 'MAG_AUTO_i'           ,#Sloan i | Kron-like elliptical aperture magnitude        [mag]  
    43: 'MAG_PETRO_i'          ,#Sloan i | Petrosian-like elliptical aperture magnitude   [mag]  
    44: 'MAG_APER_1_i'         ,#Sloan i | 5 pixel fixed aperture magnitude               [mag]  
    45: 'MAG_APER_2_i'         ,#Sloan i | 10 pixel fixed aperture magnitude              [mag]  
    46: 'MAG_APER_3_i'         ,#Sloan i | 20 pixel fixed aperture magnitude              [mag]  
    47: 'MAG_APER_4_i'         ,#Sloan i | 30 pixel fixed aperture magnitude              [mag]  
    48: 'MAGERR_AUTO_i'        ,#Sloan i | RMS error for AUTO magnitude                   [mag]  
    49: 'MAGERR_PETRO_i'       ,#Sloan i | RMS error for PETROsian magnitude              [mag]  
    50: 'MAGERR_APER_1_i'      ,#Sloan i | RMS error for MAG_APER (1)                     [mag]  
    51: 'MAGERR_APER_2_i'      ,#Sloan i | RMS error for MAG_APER (2)                     [mag]  
    52: 'MAGERR_APER_3_i'      ,#Sloan i | RMS error for MAG_APER (3)                     [mag]  
    53: 'MAGERR_APER_4_i'      ,#Sloan i | RMS error for MAG_APER (4)                     [mag]  
    54: 'FLUX_RADIUS_i'        ,#Sloan i | Fraction-of-light radii (0.5 of total)         [pixel]
    55: 'MAG_AUTO_z'           ,#Sloan z | Kron-like elliptical aperture magnitude        [mag]  
    56: 'MAG_PETRO_z'          ,#Sloan z | Petrosian-like elliptical aperture magnitude   [mag]  
    57: 'MAG_APER_1_z'         ,#Sloan z | 5 pixel fixed aperture magnitude               [mag]  
    58: 'MAG_APER_2_z'         ,#Sloan z | 10 pixel fixed aperture magnitude              [mag]  
    59: 'MAG_APER_3_z'         ,#Sloan z | 20 pixel fixed aperture magnitude              [mag]  
    60: 'MAG_APER_4_z'         ,#Sloan z | 30 pixel fixed aperture magnitude              [mag]  
    61: 'MAGERR_AUTO_z'        ,#Sloan z | RMS error for AUTO magnitude                   [mag]  
    62: 'MAGERR_PETRO_z'       ,#Sloan z | RMS error for PETROsian magnitude              [mag]  
    63: 'MAGERR_APER_1_z'      ,#Sloan z | RMS error for MAG_APER (1)                     [mag]  
    64: 'MAGERR_APER_2_z'      ,#Sloan z | RMS error for MAG_APER (2)                     [mag]  
    65: 'MAGERR_APER_3_z'      ,#Sloan z | RMS error for MAG_APER (3)                     [mag]  
    66: 'MAGERR_APER_4_z'      ,#Sloan z | RMS error for MAG_APER (4)                     [mag]  
    67: 'FLUX_RADIUS_z'        ,#Sloan z | Fraction-of-light radii (0.5 of total)         [pixel]
    68: 'KRON_RADIUS'          ,#Kron aperture in units of A_IMAGE                        
    69: 'PETRO_RADIUS'         ,#Petrosian aperture in units of A_IMAGE                    
    70: 'A_IMAGE'              ,#Profile RMS along major axis                             [pixel]  
    71: 'B_IMAGE'              ,#Profile RMS along minor axis                             [pixel] 
    72: 'THETA_IMAGE'          ,#Position angle (CCW/x)                                   [deg] 
    73: 'ELONGATION'           ,#A_IMAGE/B_IMAGE
    74: 'ELLIPTICITY'          ,#1 - B_IMAGE/A_IMAGE    
    75: 'FLAGS'                ,#Extraction flags 
   
}

def fix_headers(cat):
    for col in columns:
        cat['col{}'.format(col)].name = columns[col]
        if columns[col].startswith('MAG'):
            cat[columns[col]].unit = u.mag
        if columns[col].startswith('FLUX_RADIUS_'):
            cat[columns[col]].unit = u.pix
    cat['A_IMAGE'].unit = u.pix
    cat['B_IMAGE'].unit = u.pix
    cat['THETA_IMAGE'].unit = u.deg
    cat['ALPHA_J2000'].unit = u.deg
    cat['DELTA_J2000'].unit = u.deg
    return cat

In [ ]:
full_cat = Table()
for cat_name in file_list:
    cat = Table.read(cat_name, format='ascii')
    fix_headers(cat)
    
    cat.add_column(Column(data=np.zeros(len(cat)), name='sdss_stellarity', dtype='int32'))
    if 'ps.cat' in cat_name:
        cat['sdss_stellarity'] = 1
    
    full_cat = vstack([full_cat, cat])
    
    

In [ ]:
len(full_cat)

In [ ]:
full_cat.add_column(Column(data=np.arange(len(full_cat)), name='sdss_iac_id', dtype='int32'))

In [ ]:
full_cat[full_cat.colnames]

In [ ]:
cols = full_cat.colnames

cols.insert(0, old.pop(old.index('sdss_iac_id')))


In [ ]:
full_cat[cols].write('./data/IAC-SDSS_Herschel-Stripe-82_full.fits', overwrite=True)